In [1]:
using DIVAnd
using PyPlot
using NCDatasets
using Missings
using Interpolations
using Plots

if VERSION >= v"0.7"
    using Random
    using DelimitedFiles
    using Statistics
    using Printf
    using FileIO
else
    using Compat: @info, @warn, range, cat
end

include("../src/emodnet_bio_grid.jl");
include("../src/emodnet_bio_prep_oxy.jl");
pyplot()

LoadError: NCDatasets.NetCDFError(-49, "NetCDF: Variable not found")

In [18]:
datadir1 = "/home/ctroupin/Data/EMODnet/Chemistry/4Bio/Baltic Sea/"
filelist1 = list_data_files(datadir1);
@info "Working on $(length(filelist1)) files"
datadir2 = "/home/ctroupin/Data/EMODnet/Chemistry/4Bio/North Sea/"
filelist2 = list_data_files(datadir2);
@info "Working on $(length(filelist2)) files"

┌ Info: Working on 4 files
└ @ Main In[18]:3
┌ Info: Working on 4 files
└ @ Main In[18]:6


In [19]:
# Grid for the final re-interpolated field
field_interp = zeros(length(gridlon), length(gridlat), length(years));
xx, yy = DIVAnd.ndgrid(gridlon, gridlat);

In [44]:
# Mean fields from:
# Baltic Sea
lon1, lat1, meanfield1 = get_mean_field(filelist1, years);
# North Sea
lon2, lat2, meanfield2 = get_mean_field(filelist2, years);

In [45]:
@show size(meanfield1)
@show size(meanfield2)

size(meanfield1) = (216, 130, 7)
size(meanfield2) = (185, 142, 7)


(185, 142, 7)

In [ ]:
# Loop on the selected years
for i = 1:length(years)
    @info "Year: $(years[i])"
    # Perform interpolation
    tmp_itp = interpolate((lon, lat), meanfield[:,:,i], Gridded(Linear()));
    itp =
        @static if VERSION >= v"0.7"
            extrapolate(tmp_itp,Line())
        else
            tmp_itp
        end

    field_interp[:,:,i] = itp.(xx, yy);
end

In [4]:
write_oxy_interp(gridlon, gridlat, years, field_interp, joinpath(datadir, "oxygen_reinterp.nc"));

search: Dataset NCDatasets list_data_files @dateformat_str



```
Dataset(filename::AbstractString,mode::AbstractString = "r";
                 format::Symbol = :netcdf4)
```

Create a new NetCDF file if the `mode` is "c". An existing file with the same name will be overwritten. If `mode` is "a", then an existing file is open into append mode (i.e. existing data in the netCDF file is not overwritten and a variable can be added). With the mode set to "r", an existing netCDF file or OPeNDAP URL can be open in read-only mode.  The default mode is "r".

# Supported formats:

  * :netcdf4 (default): HDF5-based NetCDF format.
  * :netcdf4_classic: Only netCDF 3 compatible API features will be used.
  * :netcdf3_classic: classic netCDF format supporting only files smaller than 2GB.
  * :netcdf3*64bit*offset: improved netCDF format supporting files larger than 2GB.

Files can also be open and automatically closed with a `do` block.

```julia
Dataset("file.nc") do ds
    data = ds["temperature"][:,:]
end
```


In [31]:
llon = collect(10:2.:20)
llat = collect(20:.5:31.)
npoints = length(llon) * length(llat)
fieldRand = Random.rand(npoints);

In [37]:
fieldRand2 = ones(length(llon), length(llat));

In [40]:
tmp_itp = interpolate((llon, llat), fieldRand, Gridded(Linear()));

MethodError: MethodError: no method matching interpolate(::Tuple{Array{Float64,1},Array{Float64,1}}, ::Array{Float64,1}, ::Gridded{Linear})
Closest candidates are:
  interpolate(::Tuple{Vararg{Union{AbstractArray{T,1}, Tuple} where T,N}}, !Matched::AbstractArray{Tel,N}, ::IT<:Union{NoInterp, Tuple{Vararg{Union{NoInterp, Gridded},N} where N}, Gridded}) where {Tel, N, IT<:Union{NoInterp, Tuple{Vararg{Union{NoInterp, Gridded},N} where N}, Gridded}} at /home/ctroupin/.julia/packages/Interpolations/s7aNF/src/gridded/gridded.jl:61
  interpolate(!Matched::Type{TWeights}, !Matched::Type{TC}, ::Any, !Matched::IT<:Union{NoInterp, Tuple{Vararg{Union{NoInterp, BSpline},N} where N}, BSpline}) where {TWeights, TC, IT<:Union{NoInterp, Tuple{Vararg{Union{NoInterp, BSpline},N} where N}, BSpline}} at /home/ctroupin/.julia/packages/Interpolations/s7aNF/src/b-splines/b-splines.jl:103
  interpolate(!Matched::AbstractArray{#s35,1} where #s35<:Number, ::AbstractArray{TEl,1}, !Matched::TInterpolationType<:Int

In [42]:
lon1[2] - lon1[1]

0.10000038f0

In [43]:
lon2[2] - lon2[1]

0.099999905f0